In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_536211/2183090207.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [1]:
import re
from pathlib import Path 
import os
import pickle 
import pandas as pd 
import numpy as np
PATH = Path("/home/akinwilson/data/mana/quora/quora")

cols = ['creds_list', 'bio', 'profile_url']
df = pickle.load(open(PATH / "user-info.p", "rb"))[cols]
# df = df[cols]

df["creds_list"] =  (df.creds_list
                     .apply(lambda x: " ".join(x))
                     .apply(lambda x: np.nan if len(x)==0 else x)
                     .apply(lambda x: np.nan if str(x) == 'None' else x) )

df['bio'] = (df.bio
             .apply(lambda x: np.nan if len(x)==0 else x[0])
             .apply(lambda x: np.nan if str(x) == 'None' else x))

df['profile_url'] = (df.profile_url
                    .apply(lambda x: " ".join(y for y in x.split("/")[-1].split("-") if not y.isdigit())))
# column rename 
df.rename(mapper={"profile_url":"profile", "creds_list":"creds"},axis=1,inplace=True)

df.dropna(axis=0, inplace=True)


df

,creds,bio,profile
0,an American by choice. · American Ethnicity an...,.,Jae Yang
1,"MD, Internal Medicine, MS, Epidemiology, autho...",Well the steroids could decrease the efficacy ...,Amy Chai
3,stoic writer Big Data Practice Lead (2011-pres...,dad. data engineer. stoic writer. entrepreneur...,Michael David Cobb Bowen
4,Fighting the Forces of Evil (1907-present) Far...,"As someone who was once ten years old, I can t...",Thomas Oregon
6,"Analyst, Lawyer Works at U.S. Department of Ve...",Find anyone willing to buy them for more than ...,Douglas Peterson
...,...,...,...
2651,I am non-binary · LGBTQ+,you should use their proper pronouns please wh...,Saturns Fern
2652,Writer (2020-present) Studied Government in th...,"I’m a patriotic American who loves life, liber...",Lucas Quidera
2653,Works at Self-Employment Worked at Google Stud...,I was a chemist for the U.S. federal governmen...,Chuck Bluestein
2656,"BS Mechanical Engineering, University of Flori...",Graduate Engineer voted “Most Outstanding Mech...,Dean Rojas


In [2]:


# df[d]
boundaries_creds = (df.creds
                      .str
                      .len()
                      .describe(percentiles=[.05,.95])[["5%", "95%"]])

boundaries_bios = (df.bio
                  .str
                  .len()
                  .describe(percentiles=[.10,.95])[["10%", "95%"]])

df = df[(df.creds.str.len() > min(boundaries_creds)) & (df.creds.str.len() < max(boundaries_creds))]
df = df[(df.bio.str.len() > min(boundaries_bios)) & (df.bio.str.len() < max(boundaries_bios))]


txt = " ".join( df.iloc[1])

In [10]:
import spacy  # version 3.0.6'
# initialize language model

MODEL = "en_core_web_md"

nlp = spacy.load(MODEL)
# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker", last=True)

doc = nlp(txt)

def entity_linking(input_text):
    # initialize language model
    nlp = spacy.load(MODEL)
    # add pipeline (declared through entry_points in setup.py)
    nlp.add_pipe("entityLinker", last=True)
    doc = nlp(input_text)
    # returns all entities in the whole document
    all_linked_entities = doc._.linkedEntities
    # iterates over sentences and prints linked entities
    for sent in doc.sents:
        sent = sent 
        sent._.linkedEntities.pretty_print()
        sent._.linkedEntities.print_super_entities()
        print("*"*30)
    return all_linked_entities, sent 

ents,sent = entity_linking(txt)

In [16]:


sparql_hobbies = """
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr:    <http://dbpedia.org/resource/>
PREFIX dbp:    <http://dbpedia.org/property/>
PREFIX pto:     <http://www.productontology.org/id/> 

SELECT ?label ?id
WHERE  {
?page dbo:wikiPageID 31257416 .
?page dbo:wikiPageWikiLink  ?hobbyList.
 ?hobbyList rdfs:label ?label;
 dbo:wikiPageID ?id .
FILTER (LANG( ?label) = 'en')
 } 
"""

sparql_occuptations = """
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbr:    <http://dbpedia.org/resource/>
PREFIX dbp:    <http://dbpedia.org/property/>
PREFIX pto:     <http://www.productontology.org/id/> 
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?label ?id
WHERE  {
?page dbo:wikiPageID 35283370; 
dbo:wikiPageWikiLink ?ocs.
?ocs  dbo:wikiPageWikiLink ?jobs. 
?jobs rdfs:label ?label;
 dbo:wikiPageID ?id .
FILTER (LANG( ?label) = 'en')
 } 
"""



<EntityCollection (1 entities):
-https://www.wikidata.org/wiki/Q45581 Michael the Archangel     archangel in Jewish, Christian, and Islamic teachings>

In [68]:
class TextProcessor:


    def __init__(self, model_version="en_core_web_trf"):
        self.model_version = model_version
        self.nlp = spacy.load(self.model_version)
        # add pipeline (declared through entry_points in setup.py)
        self.nlp.add_pipe("entityLinker", last=True)
    
    def entity_linking(input_text):
        # initialize language model
        nlp = spacy.load("en_core_web_trf")
        # add pipeline (declared through entry_points in setup.py)
        nlp.add_pipe("entityLinker", last=True)
        doc = nlp(input_text)
        # returns all entities in the whole document
        all_linked_entities = doc._.linkedEntities
        # iterates over sentences and prints linked entities
#         for sent in doc.sents:
#             print("*"*30)
#             sent._.linkedEntities.get_label()
#             sent._.linkedEntities.print_super_entities()
#             print("*"*30)
        return all_linked_entities 


    
class Creator:
    
    business_ids = { "Q4830453": "business",
                     "Q783794": "company",
                     "Q6881511":"enterprise",
                   "Q219577": "holding company"}
    
    proffesion_ids = {"Q12737077":"occupation",
                      "Q28640"   :"profession",
                      "Q4164871" :"position",
                      "Q255274"  :"white-collar worker"}
    
    lang_ids = {"Q34770"  : "language",
                "Q1288568": "modern language"}
    
    country_ids = {"Q6256": "country"}

    '''An object representing the avaiable data asccoiated to a creator'''
    def __init__(self,short_uuid, first_name, last_name, username, headline, bio, domain, created_at, creator_status, existing_content, profile_status, **kwargs):
        self.short_uuid = short_uuid
        self.first_name = first_name
        self.last_name = last_name
        self.username = username
        self.headline = headline
        self.bio = bio
        self.domain = domain
        self.created_at = created_at
        self.creator_status =  creator_status # "PENDING" or "APPROVED"
        
        self.existing_content = existing_content
        self.profile_status = profile_status
        
        self.profession = []
        self.affiliated_business = []
        self.language = []
        self.posted_content = []
    
    def get_profession(self):
        
        headline_clean = self.text_pre_preprocessing(self.headline)
        bio_clean = self.text_pre_preprocessing(self.bio)
        
        aggregated_text = headline_clean + " " + bio_clean
        aggregated_text = " ".join(aggregated_text.split())
        text_processor = TextProcessor()
        doc = text_processor.nlp(aggregated_text)
        all_ents = doc._.linkedEntities
        
        pprint(dict([(x.get_id(), x.get_label()) for x in all_ents if x.get_label()]))
  
    def get_searches(self):
        pass
    
    def get_view_profiles(self):
        pass

    def get_viewed_sessions(self):
        pass 
    
    def get_transition_history(self):
        pass
    
    def text_pre_preprocessing(self, text):
        
        text = re.sub("(?<=\s)?ex-", "", text)
        text = re.sub("(?<=\s)[x]", "", text)        
        
        text = text.lower()
        punct= ["\\", "+", ":", ";", "@", "-", "|", "(", ")", "?", "!", "&", ".", ",", "/", "`", "'", '"']
        text = "".join([ch for ch in text if ch not in punct])

        return text

NameError: name 'user' is not defined

In [73]:
import json
import os
import spacy
from pprint import pprint
nlp = spacy.load("en_core_web_trf")


class Preproccessor:
    def __init__(self):
        self.root_path =  "" # os.path.dirname(os.path.realpath(__file__))
        
    def entity_tagging(self, doc):
        doc = nlp(doc)    
        doc_ents = []
        for ent in doc.ents:
            doc_ents.append({ent.text:ent.label_})  
        return {k: v for d in doc_ents for k, v in d.items()}
        
        
    def reduce_to(self, keys = ["id","headline", "bio"]):
        holder = []
        f = "./data/users.json"
        with open(f, "r") as file_handle:
            user = json.load(file_handle)
        for i in range(len(user)):
            reduced_vals = [user[i][k] for k in keys]  
            if None not in reduced_vals:
                if reduced_vals[1] != "" and reduced_vals[2] != "":
                    holder.append({k:v for k,v in zip(keys,reduced_vals)})    
        print("number of completed headline and bios: ", len(holder))

        return holder
    

    def entities_tagging(self, bio_headline_list):
        ents_holder = []
        for i in range(len(bio_headline_list)):
            keys = list(bio_headline_list[i].keys())
            user_id = bio_headline_list[i][keys[0]]
            ents_headline  = entity_tagging(bio_headline_list[i][keys[1]])
            ents_bio = entity_tagging(bio_headline_list[i][keys[2]])
            vals = [user_id, ents_headline, ents_bio]
            ents_holder.append({k:v for k,v in zip(keys, vals)}) 
        return ents_holder

# bh = reduce_to()
# bh_ents = entities_tagging(bh)
# pprint(list(zip(bh, bh_ents)))
Preproccessor().reduce_to()
# text = "".join([x['bio'] for x in Preproccessor().reduce_to()])

number of completed headline and bios:  175


[{'id': 1,
  'headline': 'Director of Engineering @ Mana',
  'bio': 'Career experience: Goldman Sachs, McKinsey, Man Group, NatWest Ventures, Earnd.\n\nI love building teams and software from the ground up, and I love talking about it too!'},
 {'id': 5,
  'headline': 'Chief Product Officer @ Mana',
  'bio': 'Silicon Valley expat (Apple, Amazon, Intel) with an international experience across US, Europe, Australia, India, and China.\nBobby is an actor featured in commercials (Dove for Men and Chelsea Football Club), and sang on-stage w/Taylor Swift'},
 {'id': 8,
  'headline': 'Product Strategy Lead @ Mana',
  'bio': 'Engineering Student @ UCL, Product + Ops @ Mana, Student Partner @ Entrepreneur First. I love education, entrepreneurship, enabling happy and successful remote teams, and building things.'},
 {'id': 12,
  'headline': 'CEO @Mana | xSoftBank xMcKinsey',
  'bio': 'Entrepreneur, ex-venture capitalist and strategy consultant. Here to help aspirational people reach their dreams - 

# Entity Tagging

In [143]:
class Session:
    '''Object representing avaiable session-related information'''
    def __init__(self):
        self.creator_short_uuid = None  
        self.session_uuid = None 
        self.name = None 
        self.description = None 
        pass 
    
    
# class Interaction:
#     '''Object representing an interation on Mana
#      - Viewing external content
#      - Exploring profiles
#      - Session reviews
#     '''
#     def __init__(self,):
#         pass

In [79]:
import spacy  # version 3.0.6'
# initialize language model

MODEL = "en_core_web_md"

nlp = spacy.load(MODEL)
# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entity_linker", last=True)

doc = nlp(text)

def entity_linking(input_text):
    # initialize language model
    nlp = spacy.load(MODEL)
    # add pipeline (declared through entry_points in setup.py)
    nlp.add_pipe("entity_linker", last=True)
    doc = nlp(input_text)
    # returns all entities in the whole document
    all_linked_entities = doc._.linkedEntities
    # iterates over sentences and prints linked entities
    for sent in doc.sents:
        print("*"*30)
        sent._.linkedEntities.pretty_print()
        sent._.linkedEntities.print_super_entities()
        print("*"*30)
    return all_linked_entities 


ents = entity_linking(text)

NameError: name 'text' is not defined

# Ranking text

In [ ]:
import spacy
import pytextrank

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load('en_core_web_trf')

# example text
 # text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
text = "".join([x['bio'] for x in Preproccessor().reduce_to()])


# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
doc = nlp(text)

# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print("*"*20)
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print(phrase.chunks)
    print("*"*20)

# Entity linking

In [117]:
from pprint import pprint



# 891723, 'public company'), (4830453, 'business'), (6881511, 'enterprise'

wikidata_business_ids = {"Q4830453": "business",
                         "Q783794": "company",
                         "Q6881511":"enterprise"}

wikidata_proffesion_ids = {"Q12737077":"occupation",
                            "Q28640": "profession",
                            "Q4164871": "position",
                            "Q255274": "white-collar worker"}

wikidata_lang_ids = {"Q34770": "language",
                     "Q1288568": "modern language"}

wikidata_country_ids = {"Q6256": "country"}


# pprint([((y.get_id(), y.get_label()),
#          [(x.get_id(), x.get_label()) for x in y.get_super_entities()], #  if x.get_label() in wikidata_ids.values()
#          y.get_label(), y.get_description()) 
#         for y in ents])

pprint([x.get_label() for x in ents])

['qualia',
 'Goldman Sachs',
 'McKinsey',
 'Man Group',
 'National Westminster Bank',
 'construction',
 'team',
 'software',
 'ground',
 'expatriate',
 'Apple Inc.',
 'Amazon',
 'Intel',
 'qualia',
 'United States of America',
 'European Union',
 'Australia',
 'India',
 "People's Republic of China",
 'Bobby',
 'actor',
 'radio advertisement',
 'Dove',
 'Mensa',
 'Chelsea F.C.',
 'stagecoach',
 'William Sealy Gosset',
 'UEFA Champions League',
 'Mana',
 'product',
 'Ops',
 'William Sealy Gosset',
 'Entrepreneur First',
 'education',
 'entrepreneurship',
 'team',
 'construction',
 'thing',
 'Entrepreneur',
 'Daifugō',
 'strategy',
 'human',
 'dream',
 'Dream Job',
 'company',
 'Italian',
 'year',
 'Developing',
 'business',
 'community',
 'scale',
 'YouTube',
 'Google',
 'Google',
 'YouTube',
 'YouTube',
 'YouTuber',
 'passion',
 'physical fitness',
 'entrepreneurship',
 'Best Life',
 'sacrifice',
 'A* search algorithm',
 'Unas',
 'Passive income',
 'social relation',
 'executive branch'

# Wiki data

In [126]:
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

wiki_id = "Q12737077"

def get_alias(wiki_id):
    q42_dict = get_entity_dict_from_api(wiki_id)
    return WikidataItem(q42_dict).get_aliases()
    



# # create a property representing "subclass of"
# P_SUBCLASS_OF = "P279"
# p279_dict = get_entity_dict_from_api(P_SUBCLASS_OF)
# p279 = WikidataProperty(p279_dict)

# # create a lexeme representing "bank"
# L_BANK = "L3354"
# l3354_dict = get_entity_dict_from_api(L_BANK)
# l3354 = WikidataLexeme(l3354_dict)

['occupations',
 'non-professional work activities',
 'occupation',
 'non-professional work activity']